
TCPDump is a command-line packet analyzer available on Unix-like systems (such as Linux) that allows users to capture and analyze network traffic in real-time. It can capture packets traversing a specific network interface or read packets from a saved capture file.

I suggest you to run the code either on Google Colab or use it with a Linux OS


Reminder = Google Colab runs on Linux virtual Machine , so it takes all the commands which can be used in Linux on the google cloud


Our first step is to install the required tools

In [ ]:
!apt-get install net-tools
!apt-get install tcpdump
!apt-get install iputils-ping
!apt-get install dnsutils
!apt-get install tshark
!apt-get install curl

!pip install --pre scapy[basic]
!pip install pyshark

Let's force some UDP packets in the virtual machine so we can see it in the tcpdump.

Reminder : DNS works on UDP

In [ ]:
!tcpdump -i eth0 -nn udp  & (sleep 5;dig www.polimi.it > /dev/null)

There are two packets which were the DNS REQUEST and DNS RESPONSE

Let's try capturing icmp with pinging the polimi website

In [ ]:
!tcpdump -i eth0 -nn icmp  & (sleep 5;ping www.gazzetta.it -c 10 > /dev/null)

cern website is the first website ever made
.
http://info.cern.ch
.
we can use curl to use http requests
.

curl is a command-line tool and library used to transfer data to or from a server using various protocols like HTTP, HTTPS, FTP, FTPS, and more. It supports a wide range of operations including downloading files, sending requests with custom headers, and interacting with REST APIs directly from the terminal.

In [ ]:
!curl info.cern.ch

Let's find the ip address of this website

In [13]:
import socket

def get_ip_address(hostname):
    try:
        ip_address = socket.gethostbyname(hostname)
        return ip_address
    except socket.error as e:
        print(f"Error: {e}")
        return None

hostname = 'info.cern.ch'

# Call the function to get the IP address
target = get_ip_address(hostname)

if target:
    print(f"The IP address of {hostname} is: {target}")
else:
    print(f"Failed to retrieve IP address for {hostname}")


The IP address of info.cern.ch is: 188.184.100.182


In [ ]:
!tcpdump -i eth0 -nn 'tcp and host 188.184.100.182'  & (curl -s info.cern.ch  > /dev/null)

First 3 lines are the 3 way hand shake and then there is the get request and then the 200 ok response and then the content of the website

In [ ]:
!tcpdump -i eth0 -nnX 'tcp and host 188.184.100.182'  & (sleep 5;curl -s info.cern.ch  > /dev/null)
# -nnX is used to see also the content inside the packets
# -nn just gives the headers + some info in the payload

Saving the captured packets.


The .pcap file can be downloded and opened in the wireshark application

In [16]:
!tcpdump -i eth0 -nnX 'tcp and host 188.184.100.182'  -w 'http_capture.pcap' & (sleep 5;curl -s info.cern.ch  > /dev/null)


tcpdump: listening on eth0, link-type EN10MB (Ethernet), snapshot length 262144 bytes
10 packets captured
10 packets received by filter
0 packets dropped by kernel


We can either use Scapy or Pyshark to open the .pacp file

In [ ]:
from scapy.all import *
cap = rdpcap('http_capture.pcap')
for packet in cap:
  packet.show()

# scapy is better for injecting packets than reading the packets
# its better to use pyshark


In [ ]:
!pip install nest_asyncio
import pyshark
import os
import nest_asyncio

#nest is needed in colab when using pyshark
nest_asyncio.apply()
cap = pyshark.FileCapture('http_capture.pcap')
for packet in cap :
  print(packet)
cap.close()


In [ ]:
# The loaded file is an object , so we can use filters in order to see better results that can be understood better
cap = pyshark.FileCapture('http_capture.pcap')
for packet in cap :
  print('From :' + packet.ip.src + ':' + packet.tcp.srcport + ' TO :' + packet.ip.dst + ':' + packet.tcp.dstport + ' , ' + packet.length + '  bytes  ' + packet.sniff_timestamp)
cap.close()

